# Collecte, Traitement, et Analyse de données de réseaux sociaux

Importation des librairies

In [1]:
import json
import re
import pandas as pd
from textblob import TextBlob
import random as rd

Ouverture du fichier Json et nettoyage du text

In [2]:
#Ouverture du fichier jason
with open('versailles_tweets_100.json') as json_data:
    data_dict = json.load(json_data)

#Effacer tous les caractères spéciaux dans le texte et récupérer le texte propre dans une liste
liste_txt =[]
for dict in data_dict:
    dict["text"] = re.sub(r'[\W_]+',' ', dict["text"])
    liste_txt.append(dict['text'])

# Création et écriture dans le fichier zone d'atterissage
with open("zone_d'atterrissage.json", 'w') as f2:
    json.dump(data_dict, f2)





Identification de l'auteur de la publication

In [3]:
with open('versailles_tweets_100.json') as json_data:
    data = json.load(json_data)

liste_auteurs= []
for dict in data:
    liste_auteurs.append(dict['author_id'])
print(liste_auteurs)

['1339914264522461187', '1339914264522461187', '1339914264522461187', '1339914264522461187', '717025418', '992904738516717570', '992904738516717570', '736523371', '1471684208', '992904738516717570', '3169236915', '992904738516717570', '16267684', '60117154', '3169236915', '372993152', '372993152', '105241852', '2357913366', '717025418']


Extraction de la liste des hashtags et la liste des mentions de chaque publication

In [4]:
liste_hashtags = []
liste_mentions = []

for dict in data:
    #Extraction des hashtags
    hashtags = re.findall(r'\B#\w*[a-zA-Z]+\w*', dict['text'])
    liste_hashtags.append(hashtags)
    #Extraction des mentions
    mentions = re.findall(r'\B@\w*[a-zA-Z]+\w*', dict['text'])
    liste_mentions.append(mentions)

print(liste_hashtags)

print(liste_mentions)

[['#twitter225'], ['#SupportriceMazo', '#domie', '#CIV'], ['#CIV'], ['#jifa'], [], [], [], [], ['#versailles', '#nocturne', '#appollon'], [], [], [], [], [], [], [], [], [], [], []]
[[], ['@ericbailly24', '@maxigr04del'], [], [], [], ['@isabelle170516', '@leonna_julie', '@Steiner2502'], ['@LynLyna12', '@leonna_julie'], [], [], ['@leonna_julie'], ['@miliemelo82', '@kilianbridoux', '@LeMeneec'], ['@Polo82810715', '@lrestistant73'], [], [], ['@Pauluskupa'], ['@anniemacmanus'], ['@yebbasmith', '@anniemacmanus'], [], ['@AzmiAnees3'], []]


Analyse de sentiments de la publication

In [5]:
liste_sentiments = []
with open("zone_d'atterrissage.json") as f:
   clean_data  = json.load(f)
   
for dico in clean_data:
    sentiment = TextBlob(dico['text']).sentiment 
    liste_sentiments.append(sentiment[1])
print(liste_sentiments)


[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Identification du/des topics de la publication 

In [6]:
# On va attribuer des topics aléatoirement 
liste_topics = ['sport', 'politique', 'culture', 'humour']
twitts_topics = [rd.choice(liste_topics) for i in range(len(liste_auteurs))]
print(twitts_topics)

['culture', 'humour', 'humour', 'sport', 'politique', 'politique', 'culture', 'humour', 'culture', 'sport', 'politique', 'sport', 'humour', 'humour', 'politique', 'humour', 'sport', 'humour', 'sport', 'culture']


Création du DataFrame à exploiter

In [42]:
df_exploitation = pd.DataFrame(list(zip(liste_auteurs,liste_txt, liste_hashtags, liste_mentions, liste_sentiments, twitts_topics)), columns = ['ID', 'Texte','Hashtags', 'Mentions', 'Sentiments', 'Topics'])
df_exploitation

# sauvegarder notre DataFrame dans un fichier csv
df_exploitation.to_csv("zone_d'exploitation.csv", index=False)

Top K hashtags

In [ ]:
'''création d'un dictionnaire contenant un hashtag et le nombre de fois qu'il apparaît'''
liste_hash = [x for elem in liste_hashtags for x in elem] 
def occurence_hashtag(liste):
    dic = {}
    for i in range (len(liste)):
            dic[liste[i]] = liste.count(liste[i]) 
    return dic

occurence_hashtag(liste_hash)
dico1=occurence_hashtag(liste_hash)
#print(dico1)

'''creation d'un top K hashtag'''
    # si k =3 alors affiche les 3 premiers hashtags les plus utilisés dans l'ordre --> le top 3

def topK_hashtag(dico,K):
    j=0
    dico_new={}
    for cle , valeur in sorted(dico.items(),key=lambda x:x[1]): #ranger en ordre croissant le dico
     
        dico_new[cle]=valeur
       # print(dico_new)
                                                     #la retourner en decroissant
    final=dict(reversed(list(dico_new.items())))     #pour avoir les # avec le + d'occurence en premier ds le dico
   
    while (j< K) : # affiche les k premiers elements de ce dico 
        for cle,valeur in final.items() :
            print(cle)
            j+=1
            if (j==K):
                break
           
       
topK_hashtag(dico1,3)


{'#twitter225': 1,
 '#SupportriceMazo': 1,
 '#domie': 1,
 '#CIV': 2,
 '#jifa': 1,
 '#versailles': 1,
 '#nocturne': 1,
 '#appollon': 1}

Top K utilisateurs mentionnés

In [65]:
'''création d'un dictionnaire contenant un utilisateur et le nombre de fois qu'il est mentionné'''
liste_ment = [x for elem in liste_mentions for x in elem] 
def occurence_mention(liste):
    occurences = []
    dic = {}
    for i in range (len(liste)):
            dic[liste[i]] = liste.count(liste[i]) 
    return dic

occurence_mention(liste_ment)
dico2=occurence_mention(liste_ment)

'''création d'un top k mention'''
def topK_mentions(dico,K):
    j=0
    dico_new={}
    for cle , valeur in sorted(dico.items(),key=lambda x:x[1]):
        dico_new[cle]=valeur
       
    final=dict(reversed(list(dico_new.items())))  
   
    while (j< K) : 
        for cle,valeur in final.items() :
            print(cle)
            j+=1
            if (j==K):
                break
           
       
topK_mentions(dico2,3)


{'@ericbailly24': 1,
 '@maxigr04del': 1,
 '@isabelle170516': 1,
 '@leonna_julie': 3,
 '@Steiner2502': 1,
 '@LynLyna12': 1,
 '@miliemelo82': 1,
 '@kilianbridoux': 1,
 '@LeMeneec': 1,
 '@Polo82810715': 1,
 '@lrestistant73': 1,
 '@Pauluskupa': 1,
 '@anniemacmanus': 2,
 '@yebbasmith': 1,
 '@AzmiAnees3': 1}

Top K utilisateur

In [ ]:
'''création d'un dictionnaire contenant un utilisateur et le nombre de fois qu'il a posté un tweet'''
def nbr_publis_utilisateur(liste):
    dic = {}
    for i in range (len(liste)):
            dic[liste[i]] = liste.count(liste[i]) 
    return dic

nbr_publis_utilisateur(liste_auteurs)
dico3=nbr_publis_utilisateur(liste_auteurs)

'''création d'un top k utilisateurs'''
def topK_users(dico,K):
    j=0
    dico_new={}
    for cle , valeur in sorted(dico.items(),key=lambda x:x[1]):
        dico_new[cle]=valeur
       
    final=dict(reversed(list(dico_new.items())))  
   
    while (j< K) : 
        for cle,valeur in final.items() :
            print(cle)
            j+=1
            if (j==K):
                break
           
       
topK_users(dico3,3)


{'1339914264522461187': 4,
 '717025418': 2,
 '992904738516717570': 4,
 '736523371': 1,
 '1471684208': 1,
 '3169236915': 2,
 '16267684': 1,
 '60117154': 1,
 '372993152': 2,
 '105241852': 1,
 '2357913366': 1}

In [9]:
'''création d'un dictionnaire contenant un topic et le nombre de fois qu'il apparait'''
def occurence_topic(liste):
    dic = {}
    for i in range (len(liste)):
            dic[liste[i]] = liste.count(liste[i]) 
    return dic

occurence_topic(twitts_topics)
dico4=occurence_topic(twitts_topics)


'''création d'un top k mention'''
def topK_mentions(dico,K):
    j=0
    dico_new={}
    for cle , valeur in sorted(dico.items(),key=lambda x:x[1]):
        dico_new[cle]=valeur
       
    final=dict(reversed(list(dico_new.items())))  
   
    while (j< K) : 
        for cle,valeur in final.items() :
            print(cle)
            j+=1
            if (j==K):
                break
           
       
topK_mentions(dico4,3)

{'culture': 4, 'humour': 7, 'sport': 5, 'politique': 4}

L’ensemble de tweets d’un utilisateur spécifique

In [39]:
def tweets_utilisateur():
    #création d'un dictionnaire contenant l'identifiant de l'utilsateur
    #comme clé et la liste de ses publications comme valuer
    df = df_exploitation[["ID", "Texte"]]
    ## nous avons trouvé cette méthode sur internet
    dico_tweets_utilisateur = df.groupby('ID').agg(lambda x: x.tolist()).to_dict('index')
    utilisateur = input("entrer l'id de l'utilisateur")
    if utilisateur in liste_auteurs:
        return dico_tweets_utilisateur[utilisateur]
    #problème: si on entre 2 fois un id non valide ça ne marche pas
    else:
        utilisateur = input("cet utilisateur n'existe pas, entrer un autre id d'utilisateur")
        return dico_tweets_utilisateur[utilisateur]

tweets_utilisateur()

{'Texte': ['Goumin des éléphants joueurs la même fatigue même twitter225',
  ' ericbailly24 maxigr04del mes tontons vous avez fait votre part JO prochain on ira en demi final au moins BRAVO à vous SupportriceMazo domie CIV',
  'Ah oui le sommeil là sera compliqué CIV est éliminé des JO Ahi on peut faire ça ',
  '31 juillet journée internationale de la femme africaine jifa']}

L’ensemble de tweets mentionnant un utilisateur spécifique

In [89]:
def find_mention():
    #création d'un dictionnaire contenant l'identifiant de l'utilsateur
    #comme clé et la liste de ses publications comme valeurs
    df = df_exploitation[["ID", "Texte"]]
    #print(df)
    mention = (input("entrer mention"))
    return df['ID'][df['Texte'].str.contains(mention, regex=False)]


find_mention()


16    372993152
Name: ID, dtype: object

Les utilisateurs mentionnant un hashtag spécifique

In [88]:
#fonction qui renvoie les utilisateurs mentionnant un hahshtag spécifique
##ça renvoie le texte pas les utilisateurs 
def find_hashtag ():
    df = df_exploitation[["ID", "Texte"]]
    hashtag = input("entrer hashtag")
    return df['ID'][df['Texte'].str.contains(hashtag, regex=False)]

find_hashtag()

8    1471684208
Name: ID, dtype: object